In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

import wandb
from wandb.integration.keras import WandbCallback


import carbontracker
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, LSTM, Dropout
import tensorflow as tf
import codecarbon
from codecarbon import EmissionsTracker
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Create a tracker
tracker = EmissionsTracker(project_name="tensorflow_experiment")
tracker.start()

In [ ]:
data_dir = 'data/DailyDelhiClimateTrain.csv'
df = pd.read_csv(data_dir)

In [ ]:
# Initialize a W&B run and set hyperparameters
wandb.init(
    project="MLOPS-Hackathon",   # Name of your project
    #entity="emiliewedenborg-technical-university-of-denmark",  # Replace with your W&B entity
    entity="amoal-danmarks-tekniske-universitet-dtu",
    name="temperature-forecasting-lstm",  # Name of the run

    config={
        "epochs": 10,
        "batch_size": 32,
        "learning_rate": 0.001,
        "optimizer": "adam"
    }
)

config = wandb.config


In [ ]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df.set_index('date', inplace= True)

n_cols = 1
dataset = df["meantemp"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

In [ ]:
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

In [ ]:
train_data = scaled_data[0:train_size, :]
train_data.shape

In [ ]:
# Creating a Training set with 60 time-steps
x_train = []
y_train = []
time_steps = 60
n_cols = 1

for i in range(time_steps, len(scaled_data)):
    x_train.append(scaled_data[i-time_steps:i, :n_cols])
    y_train.append(scaled_data[i, :n_cols])

In [ ]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))

In [ ]:
x_train.shape , y_train.shape

In [ ]:
model = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= ["mean_absolute_error"])

In [ ]:
model.summary()


In [ ]:
history = model.fit(
    x_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    callbacks=[WandbCallback(save_graph=False)]
)


In [ ]:
history.history.keys()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history["loss"])
plt.plot(history.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
# Stop tracker and get emissions
emissions = tracker.stop()
print(f"Emissions: {emissions} kg CO₂eq")

# Do actual predictions! 

In [ ]:
# Creating a testing set with 60 time-steps and 1 output
time_steps = 60
test_data = scaled_data[train_size - time_steps:, :]

x_test = []
y_test = []
n_cols = 1

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0:n_cols])
    y_test.append(test_data[i, 0:n_cols])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_cols))

[codecarbon INFO @ 19:06:54] Energy consumed for RAM : 0.001332 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:06:54] Delta energy consumed for CPU with constant : 0.001500 kWh, power : 360.0 W
[codecarbon INFO @ 19:06:54] Energy consumed for All CPU : 0.024006 kWh
[codecarbon INFO @ 19:06:54] Energy consumed for all GPUs : 0.000719 kWh. Total GPU Power : 11.716696556835902 W
[codecarbon INFO @ 19:06:54] 0.026057 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:06:54] 0.016477 g.CO2eq/s mean an estimation of 519.6076513294283 kg.CO2eq/year
[codecarbon INFO @ 19:06:57] Energy consumed for RAM : 0.057286 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:06:57] Delta energy consumed for CPU with constant : 0.001499 kWh, power : 360.0 W
[codecarbon INFO @ 19:06:57] Energy consumed for All CPU : 1.032205 kWh
[codecarbon INFO @ 19:06:57] Energy consumed for all GPUs : 0.487620 kWh. Total GPU Power : 11.751155246863501 W
[codecarbon INFO @ 19:06:57] 1.577111 kWh of electricity us

In [ ]:
# Get Prediction
predictions = model.predict(x_test)

In [ ]:
#inverse predictions scaling
predictions = scaler.inverse_transform(predictions)
predictions.shape

In [ ]:
#inverse y_test scaling
y_test = scaler.inverse_transform(y_test)

In [ ]:
RMSE = np.sqrt(np.mean( y_test - predictions )**2).round(2)
RMSE

In [ ]:
preds_acts = pd.DataFrame(data={'Predictions':predictions.flatten(), 'Actuals':y_test.flatten()})
preds_acts

In [ ]:
plt.figure(figsize = (16, 6))
plt.plot(preds_acts['Predictions'])
plt.plot(preds_acts['Actuals'])
plt.legend(['Predictions', 'Actuals'])
plt.show()

# Predict more than just the temp (multivariate predictions)

In [ ]:
n_cols = 4
cols = list(df.loc[:, ['meantemp', 'humidity', 'wind_speed', 'meanpressure']])
dataset = df[cols]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

In [ ]:
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

In [ ]:
train_data = scaled_data[0:train_size, :]
train_data.shape

In [ ]:
# Creating a Training set with 60 time-steps
x_train = []
y_train = []
time_steps = 60
n_cols = 4

for i in range(time_steps, len(train_data)):
    x_train.append(train_data[i-time_steps:i, :n_cols])
    y_train.append(train_data[i, :n_cols])

In [ ]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))
x_train.shape , y_train.shape

In [ ]:
model2 = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model2.compile(optimizer= 'adam', loss= 'mse' , metrics= ["mean_absolute_error"])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model2 = Sequential([
    Conv1D(filters=32, kernel_size=3, activation="relu", input_shape=(x_train.shape[1], n_cols)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation="relu"),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(n_cols)
])

model2.compile(optimizer="adam", loss="mse", metrics=["mean_absolute_error"])


c:\Users\adria\anaconda3\envs\mlops\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[codecarbon INFO @ 19:07:24] Energy consumed for RAM : 0.001499 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:07:24] Delta energy consumed for CPU with constant : 0.001500 kWh, power : 360.0 W
[codecarbon INFO @ 19:07:24] Energy consumed for All CPU : 0.027006 kWh
[codecarbon INFO @ 19:07:24] Energy consumed for all GPUs : 0.000810 kWh. Total GPU Power : 10.466572912481194 W
[codecarbon INFO @ 19:07:24] 0.029315 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:07:27] Energy consumed for RAM : 0.057452 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:07:27] Delta energy consumed for CPU with constant : 0.001500 kWh, power : 360.0 W
[codecarbon INFO @ 19:07:27] Energy consumed for All CPU : 1.035206 kWh
[codecarbon INFO @ 19:07:27] Energy consumed for all GPUs : 0.487709 kWh. Total GPU Power : 10.268703109950263 W
[codecarbon INFO @ 19:07:27] 1.580367 kWh of electricity used since the beginning.


In [ ]:
history2 = model2.fit(
    x_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    callbacks=[WandbCallback(save_graph=False)]
)


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0674 - mean_absolute_error: 0.1947
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0185 - mean_absolute_error: 0.1120
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0113 - mean_absolute_error: 0.0841
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0081 - mean_absolute_error: 0.0721
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0074 - mean_absolute_error: 0.0685
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0078 - mean_absolute_error: 0.0708
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0069 - mean_absolute_error: 0.0652
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0069 - mean_absolute_error: 0.0645
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0071 - mean_absolute_error: 0.0672
Epoch 10/100


[codecarbon INFO @ 19:07:39] Energy consumed for RAM : 0.001582 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:07:39] Delta energy consumed for CPU with constant : 0.001500 kWh, power : 360.0 W
[codecarbon INFO @ 19:07:39] Energy consumed for All CPU : 0.028506 kWh


 1/44 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0076 - mean_absolute_error: 0.0692

[codecarbon INFO @ 19:07:39] Energy consumed for all GPUs : 0.000858 kWh. Total GPU Power : 11.454824061148285 W
[codecarbon INFO @ 19:07:39] 0.030945 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 19:07:42] Energy consumed for All CPU : 1.036706 kWh
[codecarbon INFO @ 19:07:42] Energy consumed for all GPUs : 0.487759 kWh. Total GPU Power : 11.944573770743203 W
[codecarbon INFO @ 19:07:42] 1.582000 kWh of electricity used since the beginning.


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0053 - mean_absolute_error: 0.0571
Epoch 24/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0058 - mean_absolute_error: 0.0604
Epoch 25/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0052 - mean_absolute_error: 0.0560
Epoch 26/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0049 - mean_absolute_error: 0.0550
Epoch 27/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0050 - mean_absolute_error: 0.0559
Epoch 28/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0050 - mean_absolute_error: 0.0558
Epoch 29/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0051 - mean_absolute_error: 0.0564
Epoch 30/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0049 - mean_absolute_error: 0.0547
Epoch 31/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0049 - mean_absolute_error: 0.0552
Epoch 32/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0046 - mean_absolute_error: 0.0534
Epoch 33/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 

[codecarbon INFO @ 19:07:54] Energy consumed for RAM : 0.001665 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:07:54] Delta energy consumed for CPU with constant : 0.001500 kWh, power : 360.0 W
[codecarbon INFO @ 19:07:54] Energy consumed for All CPU : 0.030005 kWh
[codecarbon INFO @ 19:07:54] Energy consumed for all GPUs : 0.000912 kWh. Total GPU Power : 13.126884043877524 W
[codecarbon INFO @ 19:07:54] 0.032583 kWh of electricity used since the beginning.


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0033 - mean_absolute_error: 0.0438
Epoch 90/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0032 - mean_absolute_error: 0.0436
Epoch 91/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0032 - mean_absolute_error: 0.0436
Epoch 92/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0033 - mean_absolute_error: 0.0440
Epoch 93/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0032 - mean_absolute_error: 0.0435
Epoch 94/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0032 - mean_absolute_error: 0.0439
Epoch 95/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0032 - mean_absolute_error: 0.0434
Epoch 96/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0032 - mean_absolute_error: 0.0437
Epoch 97/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0032 - mean_absolute_error: 0.0442
Epoch 98/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0030 - mean_absolute_error: 0.0423
Epoch 99/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 

[codecarbon INFO @ 19:07:57] Energy consumed for RAM : 0.057619 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:07:57] Delta energy consumed for CPU with constant : 0.001500 kWh, power : 360.0 W
[codecarbon INFO @ 19:07:57] Energy consumed for All CPU : 1.038206 kWh
[codecarbon INFO @ 19:07:57] Energy consumed for all GPUs : 0.487814 kWh. Total GPU Power : 13.210696363001786 W
[codecarbon INFO @ 19:07:57] 1.583638 kWh of electricity used since the beginning.


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history2.history["loss"])
plt.plot(history2.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_mag = tfmot.sparsity.keras.prune_low_magnitude
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=105)
}

ModuleNotFoundError: No module named 'tensorflow_model_optimization'

[codecarbon INFO @ 19:10:12] Energy consumed for RAM : 0.058368 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:10:12] Delta energy consumed for CPU with constant : 0.001501 kWh, power : 360.0 W
[codecarbon INFO @ 19:10:12] Energy consumed for All CPU : 1.051708 kWh
[codecarbon INFO @ 19:10:12] Energy consumed for all GPUs : 0.488208 kWh. Total GPU Power : 10.317871957446144 W
[codecarbon INFO @ 19:10:12] 1.598284 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:10:24] Energy consumed for RAM : 0.002498 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 19:10:24] Delta energy consumed for CPU with constant : 0.001504 kWh, power : 360.0 W
[codecarbon INFO @ 19:10:24] Energy consumed for All CPU : 0.045013 kWh
[codecarbon INFO @ 19:10:24] Energy consumed for all GPUs : 0.001349 kWh. Total GPU Power : 9.706215955581497 W
[codecarbon INFO @ 19:10:24] 0.048860 kWh of electricity used since the beginning.


In [ ]:
# Creating a testing set with 60 time-steps and 1 output
time_steps = 60
test_data = scaled_data[train_size - time_steps:, :]

x_test = []
y_test = []
n_cols = 4

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0:n_cols])
    y_test.append(test_data[i, 0:n_cols])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_cols))

In [ ]:
x_test.shape , y_test.shape

In [ ]:
# Get Prediction
predictions = model2.predict(x_test)

In [ ]:
#inverse y_test scaling
y_test = scaler.inverse_transform(y_test)

In [ ]:
RMSE = np.sqrt(np.mean( y_test - predictions )**2).round(2)
RMSE

In [ ]:
from datetime import timedelta

In [ ]:
def insert_end(Xin, new_input):
    timestep = 60
    for i in range(timestep - 1):
        Xin[:, i, :] = Xin[:, i+1, :]
    Xin[:, timestep - 1, :] = new_input
    return Xin

In [ ]:
future = 30
forcast = []
Xin = x_test[-1 :, :, :]
time = []
for i in range(0, future):
    out = model2.predict(Xin, batch_size=5)
    forcast.append(out[0]) 
    print(forcast)
    Xin = insert_end(Xin, out[0, 0]) 
    time.append(pd.to_datetime(df.index[-1]) + timedelta(days=i))

In [ ]:
forcasted_output = np.asanyarray(forcast)   
forcasted_output = scaler.inverse_transform(forcasted_output)

In [ ]:
forcasted_output = pd.DataFrame(forcasted_output)
date = pd.DataFrame(time)
df_result = pd.concat([date,forcasted_output], axis=1)
df_result.columns = "Date", 'meantemp', 'humidity', 'wind_speed', 'meanpressure'
df_result.head()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Next 30 Days')

plt.subplot(2, 2, 1)
plt.plot(df['meantemp'])
plt.plot(df_result.set_index('Date')[['meantemp']])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Temp' ,fontsize=18)

plt.subplot(2, 2, 2)
plt.plot(df['humidity'])
plt.plot(df_result.set_index('Date')[['humidity']])
plt.xlabel('Date', fontsize=18)
plt.ylabel('humidity' ,fontsize=18)

plt.subplot(2, 2, 3)
plt.plot(df['wind_speed'])
plt.plot(df_result.set_index('Date')[['wind_speed']])
plt.xlabel('Date', fontsize=18)
plt.ylabel('wind_speed' ,fontsize=18)

plt.subplot(2, 2, 4)
plt.plot(df['meanpressure'])
plt.plot(df_result.set_index('Date')[['meanpressure']])
plt.xlabel('Date', fontsize=18)
plt.ylabel('meanpressure' ,fontsize=18)

plt.tight_layout()
plt.show()

In [ ]:
model2.save("temperature_forecasting_cnn.keras")

In [ ]:
import pathlib

converter = tf.lite.TFLiteConverter.from_keras_model(model2)


converter.optimizations = [tf.lite.Optimize.DEFAULT]    # Uncomment this line for Model 2 and Model 3

#def representative_data_gen():                          # Uncomment the following 5 lines for Model 3
#    for input_value, _ in test_batches.take(200):
#        yield [input_value]
#converter.representative_dataset = representative_data_gen
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model = converter.convert()
tflite_models_dir = pathlib.Path("")

tflite_model_file = tflite_models_dir/'model1.tflite'     # Change the filename here for Model2 and Model3!
x = tflite_model_file.write_bytes(tflite_model)



In [ ]:

print(x)

In [ ]:
#@title Run this cell each time to test your model's accuracy (make sure to change the filename)
from tqdm import tqdm
from ai_edge_litert.interpreter import Interpreter

# Load TFLite model and allocate tensors.
tflite_model_file = '/content/model1.tflite'                 # Change the filename here for Model 2 and 3
interpreter = Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(100)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))

    test_labels.append(label.numpy()[0])
    test_imgs.append(img)

# For model 1, I got 204.13 it/s
# For model 2, I got 156.91 it/s
# For model 3, I got 134.71s it/s
# Note: since the it/s will depend on the computer on which your Colab VM
#       instance is running -- we would expect it to vary a bit.

score = 0
for item in range(0,100):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1

print("Out of 100 predictions I got " + str(score) + " correct")

# Model 1 - 100 Correct
# Model 2 - 99 Correct
# Model 3 - 99 Correct
# Note: since training starts from a random intialization it would not be
#       surprising if your result is off by 1 or 2 correct.